# **Extracción de datos**

## API NEWSAPI

In [3]:
import subprocess
import sys

# Lista de librerías que deseas instalar
# En este caso, las librerías son 'schedule' para la programación de tareas y 'python-dateutil' para el manejo de fechas
libraries = ['schedule', 'python-dateutil']

# Función para instalar las librerías
def install_libraries():
    """
    Esta función recorre la lista de librerías especificadas y las instala utilizando pip.
    Utiliza el ejecutable de Python en el entorno actual para asegurarse de que las librerías
    se instalen correctamente en el entorno activo.
    """
    for library in libraries:
        try:
            # Ejecuta el comando pip install para cada librería en la lista
            subprocess.check_call([sys.executable, "-m", "pip", "install", library])
            print(f"Librería '{library}' instalada exitosamente.")
        except subprocess.CalledProcessError as e:
            # En caso de error durante la instalación de alguna librería, se captura y muestra un mensaje
            print(f"Hubo un error al intentar instalar '{library}': {e}")

# Llamar a la función para instalar las librerías
install_libraries()

# Mensaje final indicando que el proceso de instalación ha terminado
print("Proceso de instalación de librerías finalizado.")


Librerías instaladas exitosamente.


In [1]:
import requests  # Biblioteca para realizar solicitudes HTTP a servidores web.
import pandas as pd  # Biblioteca para manipulación y análisis de datos en formato tabular (DataFrame).
from datetime import datetime  # Módulo para trabajar con fechas y horas en Python.
from dateutil import parser  # Biblioteca para analizar cadenas de texto con fechas y convertirlas a objetos datetime.
import schedule  # Biblioteca para programar y ejecutar tareas periódicas a intervalos regulares.
import time  # Módulo que permite trabajar con el tiempo, incluyendo pausas en la ejecución del programa.

# Función para obtener noticias
def fetch_news():
    print("Iniciando la extracción de noticias...")  # Agregado para saber si está comenzando

    # Definir la URL de la API con parámetros
    url = 'https://newsapi.org/v2/everything'

    # Establecer los parámetros de la solicitud
    params = {
        'q': 'technology',  # Solo noticias sobre tecnología
        'sortBy': 'publishedAt',  # Ordenar por fecha de publicación
        'apiKey': 'e3abbf3202924217ae7b0beee1f689d4'  # Tu API Key
    }

    try:
        print(f"Realizando solicitud a la API con los parámetros: {params}")  # Muestra los parámetros de la solicitud
        # Realizar la solicitud a la API
        response = requests.get(url, params=params)
        response.raise_for_status()  # Verificar si la solicitud fue exitosa
        print("Solicitud a la API exitosa.")  # Confirmación de que la solicitud fue exitosa

        # Obtener los datos en formato JSON
        data = response.json()

        # Verificar si la respuesta contiene artículos
        if 'articles' in data:
            articles = data['articles']
            print(f"Se han encontrado {len(articles)} artículos.")  # Muestra la cantidad de artículos encontrados

            # Crear una lista de URLs existentes (para evitar duplicados)
            existing_urls = set()

            # Filtrar artículos y agregar solo los nuevos
            filtered_articles = []
            for article in articles:
                # Obtener fecha de publicación y convertirla
                fecha_publicacion = parser.parse(article['publishedAt'])

                # Convertir la fecha de publicación a naive si es aware
                fecha_publicacion = fecha_publicacion.replace(tzinfo=None)

                # Si no es duplicada, agregarla
                if article['url'] not in existing_urls:
                    filtered_articles.append({
                        'Título': article.get('title', 'Sin título'),
                        'Descripción': article.get('description', 'Sin descripción'),
                        'Autor': article.get('author', 'Autor no disponible'),
                        'URL': article.get('url', 'URL no disponible'),
                        'Fecha de publicación': article.get('publishedAt', 'Fecha no disponible'),
                        'Imagen': article.get('urlToImage', 'Imagen no disponible')
                    })
                    existing_urls.add(article['url'])  # Añadir URL a la lista de existentes

            # Si se encontraron artículos, convertirlos a DataFrame
            if filtered_articles:
                df_api_1 = pd.DataFrame(filtered_articles)

                # Mostrar los primeros registros del DataFrame
                print("Artículos filtrados y convertidos a DataFrame:")
                print(df_api_1.head())  # Muestra las primeras filas del DataFrame

                # Guardar el DataFrame en un archivo CSV
                df_api_1.to_csv('api_newsapi', index=False)
                print('Datos guardados en "api_newsapi".')  # Mensaje cuando los datos son guardados
            else:
                print('No se encontraron artículos recientes de tecnología.')

        else:
            print('No se encontraron artículos en la respuesta de la API.')

    except requests.exceptions.RequestException as e:
        print(f'Error al hacer la solicitud a la API: {e}')


# Llamar a la función para obtener noticias de inmediato
fetch_news()

# Programar la ejecución cada 7 días
schedule.every(7).days.do(fetch_news)

# Mantener el script en ejecución
while True:
    schedule.run_pending()
    time.sleep(1)






Iniciando la extracción de noticias...
Realizando solicitud a la API con los parámetros: {'q': 'technology', 'sortBy': 'publishedAt', 'apiKey': 'e3abbf3202924217ae7b0beee1f689d4'}
Solicitud a la API exitosa.
Se han encontrado 100 artículos.
Artículos filtrados y convertidos a DataFrame:
                                              Título  \
0  Apple CEO Tim Cook Declares New iPhone Era In ...   
1  “Placebo AI”: Is Your Business Using Automatio...   
2              Apple Forced To Cancel Beloved iPhone   
3  Chinese automobiles gaining popularity among G...   
4  Samsung and Apple's plans for thinner smartpho...   

                                         Descripción  \
0  The Apple CEO’s latest visit to London saw him...   
1  Picture the last time you called a service hot...   
2  Apple is removing the current iPhone SE from s...   
3  ACCRA, Dec. 14 (Xinhua) -- Ghanaians are incre...   
4  Samsung and Apple's plans for thinner smartpho...   

                                      

'\n# Programar la ejecución cada 12 horas\nschedule.every(12).hours.do(fetch_news)\n\n# Mantener el script en ejecución\nwhile True:\n    schedule.run_pending()\n    time.sleep(1)\n'

## API MEDIASTACK

In [2]:
import requests
import pandas as pd
import schedule
import time

# Función para obtener noticias de la API
def fetch_news():
    print("Iniciando la extracción de noticias...")  # Mensaje para indicar que la función ha comenzado

    # Definir la URL de la API con parámetros de categorías
    url = 'https://api.mediastack.com/v1/news'
    params = {
        'access_key': 'dff24c36aa79823c1effa0ff3a93f11a',  # Reemplaza con tu clave de acceso
        'categories': 'technology',  # Solo noticias sobre tecnología
    }

    try:
        # Realizar la solicitud a la API
        response = requests.get(url, params=params)
        
        # Verificar si la solicitud fue exitosa
        response.raise_for_status()
        
        # Obtener los datos en formato JSON
        data = response.json()
        
        # Verificar si la respuesta contiene artículos
        if 'data' in data:
            articles = data['data']
            print(f"Se han encontrado {len(articles)} artículos.")  # Muestra la cantidad de artículos encontrados
            
            # Convertir los artículos a un DataFrame
            df_api_2 = pd.DataFrame(articles)
            
            # Mostrar el DataFrame resultante
            print("Artículos extraídos y convertidos a DataFrame:")
            print(df_api_2.head())  # Muestra las primeras filas del DataFrame
            
            # Guardar el DataFrame en un archivo CSV
            df_api_2.to_csv('api_mediastack.csv', index=False)
            print('Datos guardados en "api_mediastack".')  # Mensaje cuando los datos son guardados
        else:
            print('No se encontraron artículos en la respuesta.')

    except requests.exceptions.RequestException as e:
        print(f'Error al hacer la solicitud a la API: {e}')


# Llamar a la función para obtener noticias de inmediato
fetch_news()

# Programar la ejecución cada 7 días
schedule.every(7).days.do(fetch_news)

# Mantener el script en ejecución
while True:
    schedule.run_pending()
    time.sleep(1)



Iniciando la extracción de noticias...
Se han encontrado 25 artículos.
Artículos extraídos y convertidos a DataFrame:
               author                                              title  \
0  Cheyenne MacDonald  Apple is reportedly trying to make a giant iPa...   
1          Simon Hill  6 Best Video Doorbell Cameras (2024): Smart, B...   
2      Natasha Singer  How Student Phones and Social Media Are Fuelin...   
3    Jordan Michelman   5 Best Carbon Steel Pans For Every Budget (2024)   
4                None  'Meu coração dava uns trancos, e um aparelho c...   

                                         description  \
0  We’ve been hearing rumors about the foldables ...   
1  Never miss a delivery. These WIRED-tested pick...   
2  Cafeteria melees. Students kicked in the head....   
3  Our expert cooked with 20 pans and these are t...   
4  'Meu coração dava uns trancos, e um aparelho c...   

                                                 url  \
0  https://www.engadget.com/mobi

'\n# Programar la ejecución cada 12 horas\nschedule.every(12).hours.do(fetch_news)\n\n# Mantener el script en ejecución\nwhile True:\n    schedule.run_pending()\n    time.sleep(1)\n'

## API CURRENTS

In [4]:
import requests
import pandas as pd
import schedule
import time

# Función para obtener noticias de la API de Currents
def fetch_news():
    print("Iniciando la extracción de noticias...")  # Mensaje para indicar que la función ha comenzado

    # Definir la URL de la API de Currents
    url = 'https://api.currentsapi.services/v1/search'
    params = {
        'apiKey': 'nIHmXWdFkHq6lDWzuvQFkABoBTTbScxztWpYiII_dCiuWLQ9',  # Reemplaza con tu clave de API
        'category': 'technology',  # Solo noticias sobre tecnología
    }

    try:
        # Realizar la solicitud a la API
        response = requests.get(url, params=params)
        
        # Verificar si la solicitud fue exitosa
        response.raise_for_status()
        
        # Obtener los datos en formato JSON
        data = response.json()
        
        # Verificar si la respuesta contiene artículos
        if 'news' in data:
            articles = data['news']
            print(f"Se han encontrado {len(articles)} artículos.")  # Muestra la cantidad de artículos encontrados
            
            # Convertir los artículos a un DataFrame
            df_api_3 = pd.DataFrame(articles)
            
            # Mostrar el DataFrame resultante
            print("Artículos extraídos y convertidos a DataFrame:")
            print(df_api_3.head())  # Muestra las primeras filas del DataFrame
            
            # Guardar el DataFrame en un archivo CSV
            df_api_3.to_csv('api_currents.csv', index=False)
            print('Datos guardados en "api_currents.csv".')  # Mensaje cuando los datos son guardados
        else:
            print('No se encontraron artículos en la respuesta.')

    except requests.exceptions.RequestException as e:
        print(f'Error al hacer la solicitud a la API: {e}')

fetch_news()

# Programar la ejecución cada 7 días
schedule.every(7).days.do(fetch_news)

# Mantener el script en ejecución
while True:
    schedule.run_pending()
    time.sleep(1)


Iniciando la extracción de noticias...
Se han encontrado 30 artículos.
Artículos extraídos y convertidos a DataFrame:
                                     id  \
0  b848a6fd-bcea-4fe6-91c4-8242c98ff413   
1  edbbd976-3e77-488f-8366-bc660e92f9fd   
2  2ed06a18-9ff5-40a4-8287-e223a15c4f94   
3  ba97c910-e870-4c87-854e-d298fa4862e4   
4  78f29e2f-7731-40b2-a392-fa2ef86f0d40   

                                               title  \
0  The PlayStation 5 laptop is a 9.5 pound portab...   
1  Why the U.S. government is saying all citizens...   
2   Intel Core Ultra 9 285K vs. Intel Core i9 14900K   
3  Xfce 4.20 Desktop Released With Wayland Improv...   
4  Roguelike ARPG Wizard of Legend 2 adds new bio...   

                                         description  \
0  Weighing in at a whopping 9.5 pounds for the l...   
1  Think twice before sending your next text mess...   
2  Intel's two latest flagships, the Core Ultra 9...   
3  After roughly two years of development the Xfc...   
4  As 

## WEBSCRAPING

#### - HACKER NEWS

In [17]:
import subprocess
import sys

def install_libraries():
    try:
        # Lista de bibliotecas a instalar
        libraries = ["requests", "beautifulsoup4"]
        
        for library in libraries:
            print(f"Instalando {library}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", library])
        print("¡Todas las bibliotecas se instalaron correctamente!")
    except Exception as e:
        print(f"Error durante la instalación: {e}")

# Llama a la función para instalar las bibliotecas
install_libraries()



Instalando requests...
Instalando beautifulsoup4...
¡Todas las bibliotecas se instalaron correctamente!


In [6]:
import subprocess  # Ejecutar comandos del sistema
import sys  # Obtener la ruta del intérprete de Python

def install_packages(packages):
    """
    Instala los paquetes proporcionados usando pip.
    """
    for package in packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"El paquete '{package}' se instaló correctamente.")
        except subprocess.CalledProcessError as e:
            print(f"Error al instalar '{package}': {e}")

# Instalar Newspaper3k y lxml[html_clean]
install_packages(["newspaper3k", "lxml[html_clean]"])

"""
- 'newspaper3k': Librería para extraer noticias de sitios web. Facilita el scraping de artículos y metadatos.
- 'lxml[html_clean]': Librería para procesar HTML, limpiando etiquetas innecesarias durante el scraping.
"""


El paquete 'newspaper3k' se instaló correctamente.
El paquete 'lxml[html_clean]' se instaló correctamente.


In [7]:
from bs4 import BeautifulSoup  # Librería para parsear y extraer información de documentos HTML
import requests  # Librería para hacer solicitudes HTTP
from newspaper import Article  # Librería para extraer artículos de noticias de URLs
import pandas as pd  # Librería para manejar y procesar datos en forma de tablas (DataFrames)

def scrape_hacker_news(pages=1):
    """
    Función para hacer scraping de noticias de Hacker News (https://news.ycombinator.com)
    
    Argumentos:
    pages -- Número de páginas a scrapear (por defecto 1).
    
    Retorna:
    all_news -- Lista con los títulos, enlaces, descripciones y autores de las noticias extraídas.
    """
    base_url = "https://news.ycombinator.com"  # URL base de Hacker News
    url = base_url  # Inicializamos la URL a la base
    all_news = []  # Lista para almacenar las noticias extraídas
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }  # Encabezados HTTP para simular una solicitud de navegador

    # Iterar sobre las páginas que se desean scrapear
    for page in range(pages):
        print(f"Scraping página {page + 1}: {url}")
        response = requests.get(url, headers=headers)  # Realizar la solicitud HTTP
        if response.status_code != 200:
            print(f"Error al acceder a la página: {response.status_code}")
            break

        soup = BeautifulSoup(response.text, "html.parser")  # Parsear el HTML de la página

        # Buscar los elementos que contienen las noticias
        news_items = soup.find_all("span", class_="titleline")

        if not news_items:
            print("No se encontraron noticias en esta página.")
            break

        # Iterar sobre cada noticia encontrada
        for item in news_items:
            link_tag = item.find("a")  # Buscar el enlace dentro del título
            if link_tag:
                title = link_tag.text.strip()  # Obtener el título de la noticia
                link = link_tag["href"]  # Obtener el enlace de la noticia
                if not link.startswith("http"):
                    link = base_url + link  # Convertir enlaces relativos a absolutos

                # Intentar generar la descripción del artículo usando 'newspaper'
                description = "No description available"
                try:
                    article = Article(link)  # Crear un objeto Article con el enlace
                    article.download()  # Descargar el contenido del artículo
                    article.parse()  # Parsear el contenido
                    article.nlp()  # Realizar análisis de texto (por ejemplo, resumen)
                    description = article.summary  # Obtener el resumen del artículo
                except Exception as e:
                    print(f"Error al extraer descripción del artículo: {e}")

                # Buscar el autor de la noticia
                parent_tr = item.find_parent("tr")  # Obtener la fila que contiene la noticia
                subline_tr = parent_tr.find_next_sibling("tr")  # Obtener la siguiente fila con detalles
                author_tag = subline_tr.find("a", class_="hnuser")  # Buscar el autor dentro del <a>
                author = author_tag.text if author_tag else "No Author"  # Obtener el nombre del autor

                all_news.append((title, link, description, author))  # Agregar la noticia a la lista

        # Buscar el enlace para la siguiente página
        more_link = soup.find("a", class_="morelink")
        if more_link:
            url = base_url + more_link["href"]  # Si hay una página siguiente, actualizar la URL
        else:
            print("No se encontró el enlace 'More'. Terminando scraping.")
            break

    return all_news  # Retornar la lista de todas las noticias extraídas

# Extraer noticias de las primeras 3 páginas
news = scrape_hacker_news(pages=3)

# Convertir los resultados a un DataFrame para su procesamiento
df = pd.DataFrame(news, columns=["Title", "Link", "Description", "Author"])

# Limpiar el DataFrame eliminando las filas sin descripción
df_clean = df[df['Description'] != "No description available"]

# Mostrar las primeras noticias, si es que existen
if not df_clean.empty:
    print(df_clean.head())  # Mostrar las primeras 5 noticias
else:
    print("No se encontraron noticias con descripción.")  # Si no hay noticias con descripción


Scraping página 1: https://news.ycombinator.com
Error al extraer descripción del artículo: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.pcmag.com/articles/nyc-wants-you-to-stop-taking-traffic-cam-selfies-but-heres-how-to-do-it on URL https://www.pcmag.com/articles/nyc-wants-you-to-stop-taking-traffic-cam-selfies-but-heres-how-to-do-it
Error al extraer descripción del artículo: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.sciencedirect.com/science/article/pii/S2666202724004518 on URL https://www.sciencedirect.com/science/article/pii/S2666202724004518
Error al extraer descripción del artículo: Article `download()` failed with HTTPSConnectionPool(host='news.ycombinator.comitem', port=443): Max retries exceeded with url: /?id=42418157 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000027DB99F8500>: Failed to resolve 'news.ycombinator.comitem' ([Errno 11001] getaddrinfo failed)")) on 

In [9]:
# Guardar el DataFrame en un archivo CSV
df_clean.to_csv('webscraping_hackernews.csv', index=False)

